* Name: Jedidah Wavinya
* ADM No: CS-DA01-25078

**Generative AI and RAG**
Objectives:
* apply your understanding of Generative AI and Retrieval-Augmented Generation (RAG) to build a practical pipeline that retrieves relevant document chunks and generates context-aware answers:

1. Step1: Import the necessary Libraries

In [ ]:
## Import Libraries
!pip install -U langchain-community
!pip install langchain faiss-cpu sentence-transformers transformers PyPDF2
!pip install pypdf
!pip install huggingface_hub
!pip install langchain-huggingface

from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFaceHub
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

2. Step2: Load the data

In [17]:
# Load the PDF
#loader = PyPDFLoader("/content/Faras_ride.pdf")
#docs = loader.load()

file_id = "1M0xW1oOkBGcKEE9dUwCwXhBWOrKtBea7"
!gdown --id {file_id} -O document.pdf

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1M0xW1oOkBGcKEE9dUwCwXhBWOrKtBea7
To: /content/document.pdf
100% 486k/486k [00:00<00:00, 73.1MB/s]


Step3: Extract text from the document

In [20]:
# Extract text from the document
text = ""
for doc in docs:
    text += doc.page_content

In [21]:
# Confirm text extraction
print(text[:500])

Faras Ride-Hailing App  
Introduction 
 Faras is a Kenyan-based ride-hailing and logistics app offering taxi, boda-boda, courier, 
and mass transport services.  
 The platform aims to simplify everyday transportation and logistics by combining 
multiple services into a single app, often referred to as a “super app.”  
 It is developed locally and targets affordability, convenience, and trust for both riders 
and drivers. 
Key Features 
 On-demand taxi and boda-boda rides 
 Courier and parce


Step4: Split into chunks

In [22]:
# Split into chunks
splitter = CharacterTextSplitter(separator="\n", chunk_size=500, chunk_overlap=50)
chunks = splitter.split_text(text)

Step5: Create embeddings in the Vector store.

In [ ]:
# Load pre-trained Sentence-Transformer
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/tmp/ipython-input-2759940654.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warni

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Step6:Store in FAISS vector DB

In [ ]:
# Store in FAISS vector DB
vectorstore = FAISS.from_texts(chunks, embedding_model)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Step7: Save vectorstore for future use

In [ ]:
# Save vectorstore for future use
vectorstore.save_local("faiss_index")

Step8: Add the Load Language Model (LLM)

In [ ]:
model_name = "google/flan-t5-large"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
flan_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
from langchain.chains import RetrievalQA
from langchain_core.language_models.llms import LLM
from langchain_community.llms import HuggingFacePipeline

# Create LLM from pipeline
llm = HuggingFacePipeline(pipeline=flan_pipeline)

# Create retriever from vectorstore
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# Create RetrievalQA chain
rag_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=True)

/tmp/ipython-input-2395286076.py:6: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=flan_pipeline)


Step9: Query Using RAG

In [ ]:
query = "What is the main purpose of the document?"
response = rag_chain(query)

print("Answer:\n", response['result'])
for doc in response['source_documents']:
    print(doc.metadata)

/tmp/ipython-input-1513495074.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = rag_chain(query)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Answer:
 a) Low commission We, at FARAS company, have societal objectives. One of our main objectives is to promote our partners livelihood conditions by taking least commission in the industry. b) better relationship with drivers We listen to you and consider your advice. We corporate with you in different channels. c) On Time SupportWe have dedicated support team whose main task is to help our partners by 24/7. Contact us: Collaboration and Advertising be a pioneer of the continent’s ride-hailing economy and on demand services FARAS is expanding services across its platform to include mass transportation, courier services, Bus Booking and E-payments to become the continent’s everyday Super App. Faras Ride-Hailing App Introduction  Faras is a Kenyan-based ride-hailing and logistics app offering taxi, boda-boda, courier, and mass transport services.  The platform aims to simplify everyday transportation and logistics by combining multiple services into a single app, often referred to a

Step10: Compare With Non-RAG Answer

In [ ]:
prompt = f"Answer this question without using the document context: {query}"
no_context_response = llm(prompt)
print("Generic Answer:\n", no_context_response)

/tmp/ipython-input-2478441682.py:2: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  no_context_response = llm(prompt)


Generic Answer:
 To describe the history of the United States of America


Step11: Create and Query the `qa_chain`

In [23]:
from langchain.chains import RetrievalQA

# Create RetrievalQA chain using the retriever and LLM
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# Ask a question using the qa_chain
question = "Where can i get the Faras App?"
result = qa_chain({"query": question})
print(result["result"])

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Android (Riders): Faras Rider on Play Store  Android (Drivers): Faras Captain on Play Store  iOS (Riders): Faras on Apple Store
